In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.5 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=37a6a7d7174b722cbeee1636f9c3ee93e5a463fa4dedcd905bb36

In [3]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import geemap.foliumap as geemap
import ee
import IPython.display as disp
import folium 
from datetime import date
#import geojson
import PIL as image

In [4]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=u-UiZvPhvr8cLYhsJkataYv4mqiMz0amip1kb_oOGPg&tc=jlMv9HSw9qEZzPF0avmvSqzQgChoa2opqwY_i5W4d50&cc=2tS3elcIhz4hbD2srzj1saFn6_04DR3mzL2wbKUnLig

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk6y2DcH7OAC0ldFyjhiaN1XEuLL2xHCAjWpy4YTIQJkmfIOv8q4u9Q

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
## Data range
start_date = '2000-01-01'
end_date = '2022-12-31'

In [7]:
Map = geemap.Map()

kerwa_lake_bhopal = ee.FeatureCollection('users/ferreirawg/kerwa_lake_bhopal')
geometry = ee.Geometry.Point([77.372411,23.167912]) 
image = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(geometry) \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20)) \
        .first()\
        .clip(kerwa_lake_bhopal)
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B8'])

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1= ndwiMasked.toInt()
vectors = ndwiMasked1.reduceToVectors(scale = 30.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)

In [8]:
Map.addLayer(kerwa_lake_bhopal, {}, 'Kerwa Lake Bhopal')
Map.addLayer(ndwi, {}, 'NDWI')
Map.addLayer(ndwiMasked1, {'palette': ['00FFFF', '0000FF']}, 'Water')
Map.centerObject(geometry, zoom=12)

In [9]:
#Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
            .filterDate(start_date,end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
            .median()\
            .clip(kerwa_lake_bhopal)


mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')

ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')


ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')

dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')

In [10]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(mndwi, {'min': -1, 'max': 1, 'palette': ['white', 'black', 'blue']}, 'MNDWI')

In [11]:
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ph, {'min': 6, 'max': 9, 'palette': ['red', 'green', 'purple']}, 'pH')

Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndti, {'min': -1, 'max': 1, 'palette': ['black', 'yellow', 'red']}, 'NDTI')

Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(ndci, {'min': -1, 'max': 1, 'palette': ['white', 'black','yellow']}, 'NDCI')


col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate(start_date,end_date) \
.filterBounds(vectors).median()\
.clip(kerwa_lake_bhopal)
temp  = col.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')

Map.addLayer(temp, {'min': -30, 'max': 30, 'palette': ['red', 'white', 'black']}, 'Temperature')

In [12]:
Map.addLayerControl()
Map